# Getting the Environment Ready

## Installation

In [1]:
%pip install python-chess
%pip install svglib
%pip install cairosvg
%pip install pdf2image

# !apt-get install poppler-utils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## importing Packages

In [1]:
import re
import os
import chess
import pickle
import sklearn
import imageio
import datetime
# import cairosvg

import numpy as np
import pandas as pd
import seaborn as sns
import chess.pgn as pgn
import matplotlib.pyplot as plt

from time import sleep
from sklearn import metrics
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from pdf2image import convert_from_path
from IPython.display import display, HTML, clear_output

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

# from sklearn.model_selection import train_test_split

## Define

In [2]:
loading_data = False
training_models = False
BOARD_SIZE = (8, 8, 6)
PIECE_TO_INDEX = {'P': 0, 'R': 1, 'N': 2, 'B': 3, 'Q': 4, 'K': 5}
CHESSY_TO_Y = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
INDEX_TO_PIECE = {0: 'P', 1: 'R', 2: 'N', 3: 'B', 4: 'Q', 5: 'K'}
Y_TO_CHESSY = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
ARRAY_PIECE = ['P','R','Q','K','B','N']
BOOLEAN_PIECE = {'P': False, 'R': False, 'Q': False,
                 'K': False, 'B': False,'N': False}
PIECE_TO_WORD = {'P': 'Pawn', 'Q': 'Queen', 'K': 'King',
                 'N': 'Knight', 'R': 'Rook', 'B': 'Bishop'}

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# Helper Functions


In [4]:
def initialize_board():
    board = np.zeros(BOARD_SIZE)
    pattern = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']

    # Putting the black non-pawn pieces
    for index, piece in enumerate(pattern):
        board[0, index, PIECE_TO_INDEX[piece]] = -1

    # Putting the black pawns
    board[1, :, PIECE_TO_INDEX['P']] = -1

    # Putting the white non-pawn pieces
    for index, piece in enumerate(pattern):
        board[BOARD_SIZE[0] - 1, index, PIECE_TO_INDEX[piece]] = 1

    # Putting the white pawns
    board[BOARD_SIZE[0] - 2, :, PIECE_TO_INDEX['P']] = 1

    return board

In [5]:
def convert_bitboard_to_image(board):
    im2d = np.array(list(str(board).replace('\n', '').replace(' ', ''))).reshape((8, 8))
    im = np.zeros(BOARD_SIZE)

    for i in range(BOARD_SIZE[0]):
        for j in range(BOARD_SIZE[1]):
            piece = im2d[i, j]
            if piece == '.': continue
            if piece.isupper():
                im[i, j, PIECE_TO_INDEX[piece.upper()]] = 1
            else:
                im[i, j, PIECE_TO_INDEX[piece.upper()]] = -1

    return im


def convert_image_to_bitboard(im):
    board = chess.Board()
    board.clear()
    for i in range(BOARD_SIZE[0]):
        for j in range(BOARD_SIZE[1]):
            index_piece = np.where(im[(i, j)] != 0)
            index_piece = index_piece[0]
            new_coords = flatten_coord2d((7 - i, j))
            if index_piece.shape != (0,):
                piece = INDEX_TO_PIECE[index_piece[0]]
                if im[(i, j, index_piece[0])] == -1:
                    piece = piece.lower()
                board.set_piece_at(new_coords, chess.Piece.from_symbol(piece))

    return board

In [6]:
def flatten_coord2d(coord2d):
    return (8 * coord2d[0]) + coord2d[1]

In [7]:
def flip_image(im):
    return im[::-1, :, :]


def flip_color(im):
    indices_white = np.where(im == 1)
    indices_black = np.where(im == -1)
    im[indices_white] = -1
    im[indices_black] = 1
    return im


def flip_coord2d(coord2d):
    return 8 - coord2d[0] - 1, coord2d[1]

In [8]:
def coord2d_to_chess_coord(coord2d):
    chess_coord = Y_TO_CHESSY[coord2d[1]] + str(8 - coord2d[0])
    return chess_coord


def chess_coord_to_coord2d(chess_coord):
    return 8 - int(chess_coord[1]), CHESSY_TO_Y[chess_coord[0]]

In [9]:
def clip_pieces(prob_dists, ims):
    mask = ims > 0
    rotated_mask = np.rollaxis(mask, 3, 1)  # Convert to form (num_samples, C, H, W)

    or_channels = np.any(rotated_mask, axis=1)
    filter = or_channels.reshape(or_channels.shape[0], 64)

    prob_dists_filtered = prob_dists * filter
    return prob_dists_filtered


def clip_move(prob_dist, im, coord, piece_type=None, castle_rights={'Q':False,'K':False}):
    board = convert_image_to_bitboard(im)
  
    if castle_rights['K']:
      board.castling_rights |= chess.BB_A1  
  
    if castle_rights['Q']:
      board.castling_rights |= chess.BB_H1  

    piece_coord = coord2d_to_chess_coord(coord)
    for i in range(BOARD_SIZE[0]):
        for j in range(BOARD_SIZE[1]):
            to_coord = coord2d_to_chess_coord((i, j))
            if piece_coord == to_coord:
                prob_dist[i * 8 + j] = 0
                continue
            uci_move = piece_coord + to_coord

            if piece_type == 'P':
                if uci_move[3:] == '8':
                    uci_move += 'q'

            if chess.Move.from_uci(uci_move) not in board.legal_moves:
                prob_dist[i * 8 + j] = 0

    return prob_dist

In [10]:
def load_pgn_games(file_name, num_games, flag):
    games = []
    pgn_file = open(file_name)
    for _ in range(num_games):
        game = pgn.read_game(pgn_file)
        if game is None:
            break
        if flag:
            games.append(game)
    if flag:
        pgn_file.close()
        return games

    # Read the second num_games
    for _ in range(num_games):
        game = pgn.read_game(pgn_file)
        if game is None:
            break
        games.append(game)

    pgn_file.close()
    return games

## Helper functions

In [12]:
pgn_file_path = "Chess_data/pgn_files/"
gif_file_path = "Chess_data/games_gif/"
model_path = "G:/My Drive/Chess_data/models/"
temp_path = "output_images/"

In [13]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [14]:
def generate_pgn_file(moves, filename):
    game = chess.pgn.Game()
    board = chess.Board()
    game.setup(board)

    for move in moves:
        board.push(move)

    node = game
    for move in board.move_stack:
        node = node.add_variation(move)

    with open(filename, "a"):
        os.utime(filename, None)
        
    with open(filename, "w") as pgn_file:
        pgn_file.write(str(game))

In [15]:
def save_svg(svg, filepath):
    try:
        file_handle = open(filepath, 'w')
    except IOError as e:
        print(str(e))
        exit(1)

    file_handle.write(svg)
    file_handle.close()

if not os.path.exists(temp_path):
    os.makedirs(temp_path)

In [16]:
def convert_all(PATH):
    images = []
    for file_name in sorted(os.listdir(PATH)):
        if file_name.endswith('.svg'):
            file_path = os.path.join(PATH, file_name)
            convert_svg_to_pdf(file_path)
            pdf_name = file_path[:-3]+"pdf"
            convert_pdf_to_png(pdf_name)
    print("Finished converting all files to png")


def convert_svg_to_pdf(input_file):
    cairosvg.svg2pdf(url=input_file, write_to=input_file[:-3]+"pdf")


def convert_pdf_to_png(pdf_path):
    images = convert_from_path(pdf_path)
    for index, image in enumerate(images):
        image.save(pdf_path[:-3]+"png")

# Run API

In [17]:
def load_piece_model(piece_label):
    PATH = model_path + "chess_model_CNN_" + piece_label + ".h5"
    loaded_model = load_model(PATH, compile=False)
    return loaded_model
model_path = "G:/My Drive/Chess_data/models/"

In [18]:
base_model1 = load_piece_model('' + "BASE" + '2')
move_models1 = {}
for piece in ARRAY_PIECE:
    move_models1[piece] = load_piece_model('' + PIECE_TO_WORD[piece] + '2')

model_no_tune = {'base_model':base_model1, 'move_models':move_models1}

In [19]:
base_model2 = load_piece_model('after_puzzles_' + "BASE" + '2')
move_models2 = {}
for piece in ARRAY_PIECE:
    move_models2[piece] = load_piece_model('after_puzzles_' + PIECE_TO_WORD[piece] + '2')

model_with_tune = {'base_model':base_model2, 'move_models':move_models2}

In [20]:
base_model3 = load_piece_model('after_puzzles_freeze_' + "BASE" + '2')
move_models3 = {}
for piece in ARRAY_PIECE:
    move_models3[piece] = load_piece_model('after_puzzles_freeze_' + PIECE_TO_WORD[piece] + '2')

model_with_freeze = {'base_model':base_model3, 'move_models':move_models3}

In [21]:
models = {'tune':model_with_tune, 'no_tune':model_no_tune, 'freeze':model_with_freeze}

In [22]:
def predict_k_move(board, model_name, k, k_from):
    if board.turn:
      im = convert_bitboard_to_image(board)
    else:
      im = flip_image(convert_bitboard_to_image(board))
      im = flip_color(im)

    img = im.reshape(1, 8, 8, 6)
    from_move = models[model_name]['base_model'].predict(img, verbose = 0)
    from_move = clip_pieces(from_move, img)
    from_move_short = [[i, from_m] for i, from_m in enumerate(from_move[0])]
    from_move_short = sorted(from_move_short, key=lambda x: x[1], reverse=True)
    from_move_short = from_move_short[:k_from]
    
    best_moves = []
    for index, prob in from_move_short:
      if prob > 0:
        from_cell_coord = [index // 8, index % 8]
        pieceType = INDEX_TO_PIECE[np.argmax(img[:, from_cell_coord[0], from_cell_coord[1]])]
        to_move = models[model_name]['move_models'][pieceType].predict(img, verbose = 0)

        castling_rights = {'K':board.has_kingside_castling_rights(color=chess.WHITE),
                            'Q':board.has_queenside_castling_rights(color=chess.WHITE)}

        to_move = clip_move(to_move[0], im, from_cell_coord,
                            pieceType, castling_rights)

        for index2, prob2 in enumerate(to_move):
          if prob2 > 0:
            from_move_index = chess.square_mirror(index) if board.turn else index
            to_move_index = chess.square_mirror(index2) if board.turn else index2
            move_from_to = chess.square_name(from_move_index) + chess.square_name(to_move_index)

            if pieceType == 'P':
              if board.turn:
                if move_from_to[3:] == '8':
                  move_from_to += 'q'
              else:
                if move_from_to[3:] == '1':
                  move_from_to += 'q'
            best_moves.append([prob * prob2, move_from_to])

    sorted_best_moves = sorted(best_moves, key=lambda x: x[0], reverse=True)

    best_k_moves_with_prob = sorted_best_moves[:k]

    best_k_moves = [element[1] for element in best_k_moves_with_prob]

    return best_k_moves

In [23]:
MAX_CACHE_SIZE = 1000000
EVICTION_BATCH_SIZE = 100
cache_called = 0
cache_hit = 0
k_move_cache = {}
cache_usage_queue = []

def predict_k_move_cached(board, model_name, k, k_from):
    global cache_hit
    global cache_called 

    fen_hash = hash(board.fen())
    cache_called += 1
    if fen_hash in k_move_cache:
        cache_hit += 1
        cache_usage_queue.remove(fen_hash)
        cache_usage_queue.insert(0, fen_hash)
        return k_move_cache[fen_hash]
    
    k_moves = predict_k_move(board, model_name, k, k_from)
    
    if len(k_move_cache) >= MAX_CACHE_SIZE:
        evicted_hashes = cache_usage_queue[-EVICTION_BATCH_SIZE:]
        for evicted_hash in evicted_hashes:
            k_move_cache.pop(evicted_hash)
            cache_usage_queue.remove(evicted_hash)
    
    k_move_cache[fen_hash] = k_moves
    cache_usage_queue.insert(0, fen_hash)
    
    return k_moves

In [24]:
def stockfish_evaluation(board, time_limit = 2):
    with chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-avx2.exe") as engine:
        info = engine.analyse(board, chess.engine.Limit(time=time_limit,depth=15),game=object())
        evaluation = info["score"].relative
        if evaluation.is_mate():
            mate_sign = str(info["score"].relative)[1:][:1]
            mate = int(str(info["score"].relative)[2:])
            return mate, (1 if mate_sign == '+' else -1)
        else:
            return evaluation.cp, 0

In [25]:
def evaluate(board, time_limit):
    eval, mate = stockfish_evaluation(board, time_limit)
    if not board.turn:
        eval = -eval
        mate = -mate
    if mate == 1:
        return float('inf') 
    if mate == -1:
        return float('-inf')
    return eval

In [26]:
def alpha_beta(board, model_name, depth, alpha, beta, k, k_from, eval_time_limit):
    if depth == 0 or board.is_game_over():
        return evaluate(board, eval_time_limit)

    best_k_move = predict_k_move_cached(board, model_name, k, k_from)
    eval_move = []
    for move in best_k_move:
        board.push(chess.Move.from_uci(move))
        eval_move.append((evaluate(board, eval_time_limit) , move))
        board.pop()
    eval_move = sorted(eval_move , key=lambda x: x[0], reverse=board.turn)

    if board.turn:
        for _,move in eval_move:
            board.push(chess.Move.from_uci(move))
            alpha = max(alpha, alpha_beta(board, model_name, depth - 1, alpha, beta, k, k_from, eval_time_limit))
            board.pop()
            if beta <= alpha:
                break  # Beta cut-off
        return alpha
    else:
        for _,move in eval_move:
            board.push(chess.Move.from_uci(move))
            beta = min(beta, alpha_beta(board, model_name, depth - 1, alpha, beta, k, k_from, eval_time_limit))
            board.pop()
            if beta <= alpha:
                break  # Alpha cut-off
        return beta

In [27]:
def alpha_beta_limit(board, model_name, depth, alpha, beta, k, idx, time, start):
    now = datetime.datetime.now()
    if (now - start).total_seconds() > time:
        return None
    if depth == 0 or board.is_game_over():
        return evaluate(board)

    best_k_move = predict_k_move_cached(board, model_name, k)[:idx]
    eval_move = []
    for move in best_k_move:
        board.push(chess.Move.from_uci(move))
        eval_move.append((evaluate(board) , move))
        board.pop()
    eval_move = sorted(eval_move , key=lambda x: x[0], reverse=board.turn)

    if board.turn:
        for _,move in eval_move:
            board.push(chess.Move.from_uci(move))
            value = alpha_beta_limit(board, model_name, depth - 1, alpha, beta, k, idx, time, start)
            if value == None:
              return None
            alpha = max(alpha, value)
            board.pop()
            if beta <= alpha:
                break  # Beta cut-off
        return alpha
    else:
        for _,move in eval_move:
            board.push(chess.Move.from_uci(move))
            value = alpha_beta_limit(board, model_name, depth - 1, alpha, beta, k, idx, time, start)
            if value == None:
              return None
            beta = min(beta, value)
            board.pop()
            if beta <= alpha:
                break  # Alpha cut-off
        return beta

In [28]:
def find_best_move(board, model_name, d, k, k_from, eval_time_limit):
    moves = predict_k_move_cached(board, model_name, k, k_from)
    eval_move = []
    for move in moves:
        board.push(chess.Move.from_uci(move))
        eval_move.append((evaluate(board, eval_time_limit) , move))
        board.pop()
    eval_move = sorted(eval_move , key=lambda x: x[0], reverse=board.turn)
    print(eval_move)
    best_move = None
    best_value = float('-inf')
    best_value2 = float('inf')

    for _, move in eval_move:
        board.push(chess.Move.from_uci(move))
        value = alpha_beta(board, model_name, d - 1, float('-inf'), float('inf'), k, k_from, eval_time_limit)
        print(value, move)
        board.pop()
        if board.turn:
            if value > best_value:
                best_value = value
                best_move = move
        else:
            if value < best_value2:
                best_value2 = value
                best_move = move
        if best_move == None:
            best_move = move
    return best_move

In [29]:
def find_best_move_limit(board, model_name, d, k, time):
    start = datetime.datetime.now()
    
    best_move = None
    for idx in range(1, k + 1):
        best_move2 = None
        best_value = float('-inf')
        best_value2 = float('inf')
        moves = predict_k_move_cached(board, model_name, k)[:idx]
        eval_move = []

        for move in moves:
            board.push(chess.Move.from_uci(move))
            eval_move.append((evaluate(board) , move))
            board.pop()

        eval_move = sorted(eval_move , key=lambda x: x[0], reverse=board.turn)
        print(eval_move)
        

        for _, move in eval_move:
            board.push(chess.Move.from_uci(move))
            value = alpha_beta_limit(board, model_name, d - 1, float('-inf'), float('inf'),k , idx, time, start)
            print(value, move)
            if value == None:
              break
            board.pop()
            if board.turn:
                if value >= best_value:
                    best_value = value
                    best_move2 = move
            else:
                if value <= best_value2:
                    best_value2 = value
                    best_move2 = move
        if not best_move2 == None:
            best_move = best_move2
    return best_move

In [30]:
def count_pieces(board):
    piece_counts = {
        chess.WHITE: 0,
        chess.BLACK: 0
    }

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            piece_counts[piece.color] += 1

    return piece_counts

In [31]:
DEPTH = 3
K_BEST = 3
K_FROM = 6
EVAL_TIME_LIMIT = 0.1
NOM_D_K = {0:(1,1), 5: (2,2), 10:(3,3)}
NOM_ARR = [10, 5, 0]

def decide(board, model_name, k_from, eval_time_limit, basic=False):
    if basic:
        if model_name2 == 'tree':
            if board.fullmove_number > 10:
                print('hi im thinking')
                best_move = find_best_move(board, 'tune', DEPTH, K_BEST, K_FROM, EVAL_TIME_LIMIT)
            else:
                best_move = predict_k_move_cached(board, 'tune', K_BEST, K_FROM)[0]
        else:
            best_move = predict_k_move_cached(board, model_name, K_BEST, K_FROM)[0]
        return best_move
    
    if model_name == 'tree':
        if count_pieces(board)[board.turn] < 5:
            best_move = find_best_move(board, 'tune', 4, 4, k_from, eval_time_limit)
        for nom in NOM_ARR:
            if(board.fullmove_number >= nom):
                best_move = find_best_move(board, 'tune', NOM_D_K[nom][0], NOM_D_K[nom][1], k_from, eval_time_limit)
                break
    else:
        best_move = predict_k_move_cached(board, model_name, K_BEST, k_from)[0]
    return best_move

In [32]:
import flask
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
app=Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


@app.route('/predict', methods = ['POST'])
@cross_origin()
def result():
    fen = request.values.get('fen')
    print(fen)
    model_name = request.values.get('model')
    board = chess.Board()
    board.set_fen(fen)
    start = datetime.datetime.now()
    best_move = decide(board, model_name, K_FROM, EVAL_TIME_LIMIT)
    end = datetime.datetime.now()
    print(best_move)
    print((end - start).total_seconds())
    print('cache_hit', cache_hit)
    print('cache_called', cache_called)
    return jsonify(best_move)

In [33]:
app.run(port='3200', host='0.0.0.0', debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3200
 * Running on http://10.2.0.2:3200
Press CTRL+C to quit


rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1


192.168.97.94 - - [16/Aug/2023 10:58:06] "POST /predict HTTP/1.1" 200 -


c7c5
1.278058
cache_hit 0
cache_called 1
rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2


...

c6b6
0.905701
cache_hit 471
cache_called 5153
4rrk1/5pp1/1q6/8/RR3P2/4p1P1/4Q2K/8 b - - 3 36


192.168.85.140 - - [23/Aug/2023 16:34:43] "POST /predict HTTP/1.1" 200 -


b6g6
0.868007
cache_hit 471
cache_called 5154
4rrk1/5pp1/6q1/8/R2R1P2/4p1P1/4Q2K/8 b - - 5 37


192.168.85.140 - - [23/Aug/2023 16:34:49] "POST /predict HTTP/1.1" 200 -


g6e6
0.924936
cache_hit 471
cache_called 5155
